In [11]:
import rospy, baxter_commander, json
import numpy as np
import tf
import time
from scipy.optimize import minimize
import transformations
import rospkg
import yaml

In [12]:
rospy.init_node('vrep_calibration')

In [13]:
rospack = rospkg.RosPack()

In [14]:
right = baxter_commander.ArmCommander('right')

Unknown tag: materialUnknown tag: materialScalar element defined multiple times: collisionScalar element defined multiple times: collision

# Defintion of Functions

### Tools

In [15]:
def random_transforms(pos_bounds, rot_bounds, nb_transforms=2):
    flat_transforms = []
    for i in range(nb_transforms):
        # add a random position within bounds
        flat_transforms += np.random.uniform(pos_bounds[0], pos_bounds[1], 3).tolist()
        # add a random quaternion
        rot = np.random.uniform(rot_bounds[0], rot_bounds[1], 4)
        flat_transforms += (rot / np.linalg.norm(rot)).tolist()
    return flat_transforms

In [16]:
def extract_transforms(flat_transforms):
    # a transform is 3 pos and 4 rot
    nb_transform = len(flat_transforms) / 7
    list_transforms = []
    for i in range(nb_transform):
        pose = []
        # extract the pose
        pose.append(flat_transforms[i * 7:i * 7 + 3])
        pose.append(flat_transforms[i * 7 + 3:i * 7 + 7])
        # append it to the list of transforms
        list_transforms.append(pose)
    return list_transforms

### Cost function for the optimization

In [17]:
def distance_cost(pose1, pose2, rot_coeff=2):
    pos_cost = 0
    # calculate position ditance
    pos_cost = np.linalg.norm(np.array(pose1[0]) - np.array(pose2[0]))
    # distance between two quaternions
    rot_cost = 1 - np.inner(pose1[1], pose2[1])**2
    return pos_cost + rot_coeff * rot_cost

In [18]:
def quaternion_cost(list_calibr, norm_coeff):
    C = 0
    for transform in list_calibr:
        # norm of a quaternion is always 1
        C += norm_coeff * abs(np.linalg.norm(transform[1]) - 1)
    return C

In [19]:
def evaluate_calibration(calibrations, tf_robot, tf_vrep):
    # first extract the transformations
    list_calibr = extract_transforms(calibrations)
    # collect the cost based on the quaternions
    cost = quaternion_cost(list_calibr, 0.5)
    # loop trough all the tf
    for i in range(len(tf_robot)):
        # compute the corresponding transformation from recorded data
        pose = transformations.multiply_transform(list_calibr[0], tf_vrep[i])
        pose = transformations.multiply_transform(pose, list_calibr[1])
        # compute the cost based on the distance
        cost += distance_cost(tf_robot[i], pose)

    print cost
    return cost

## Record Points

In [23]:
points = {"tf": [], "joints":[]}
tf_robot = []
while True:
    entry = raw_input('Press Enter to record a new point or Q-Enter to exit: ')
    if entry!='':
        break
    fk = right.endpoint_pose()
    joints = baxter_commander.persistence.statetodict(right.get_current_state())
    tf_robot.append([fk[0], fk[1]])
    points["tf"].append(fk[0]+fk[1])
    points["joints"].append(joints["position"])

Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: 
Press Enter to record a new point or Q-Enter to exit: q


In [21]:
points = {"tf": [], "joints":[]}
tf_robot = []
for i in range(10):
    goal = right.get_random_pose()
    # display only first
    right.move_to_controlled(goal, display_only=True)
    entry = raw_input('Is the trajectory correct ? y/[N] ')
    if entry == 'y':
        # now execute the trajectory
        right.move_to_controlled(goal)
        fk = right.endpoint_pose()
        joints = baxter_commander.persistence.statetodict(right.get_current_state())
        tf_robot.append([fk[0], fk[1]])
        points["tf"].append([fk[0], fk[1]])
        points["joints"].append(joints["position"])

Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] y
Is the trajectory correct ? y/[N] y
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 
Is the trajectory correct ? y/[N] 


In [24]:
with open('/tmp/robot_joints.json', 'w') as f:
    json.dump(points, f, indent=4)

## Matlab part: Get VREP coordinates with the given joint angles

In [25]:
with open('/tmp/vrep_transform.json') as f:
    data = json.load(f)
# convert the format to get transform matrices
positions = np.array(data['cartesian_XYZ'])
quaternions = np.array(data['quaternion_XYZScale'])
tf_vrep = []
for i in range(len(tf_robot)):
    pos = positions[i]
    rot = quaternions[i]
    tf_vrep.append([pos, rot])

## Optimization to find the calibration matrices

In [26]:
# set limits for search space
bounds = []
pos_bounds = [-0.5, 0.5]
rot_bounds = [-1, 1]
initial_guess = random_transforms(pos_bounds, rot_bounds, 2)

In [27]:
t0 = time.time()
result = minimize(evaluate_calibration, initial_guess, args=(tf_robot, tf_vrep, ))
print time.time()-t0, "seconds of optimization"

21.5408899111
21.5408898726
21.540889957
21.5408898132
21.5408901018
21.540890036
21.5408896806
21.5408896897
21.5408898857
21.5408898378
21.5408899085
21.5408900176
21.5408899732
21.540890044
21.5408899569
21.5408899111
113.487520321
113.487520321
113.487520334
113.487520292
113.487520444
113.487520331
113.487520322
113.487520329
113.487520334
113.487520328
113.487520367
113.487520281
113.487520318
113.487520321
113.487520312
113.487520323
47.8532761838
47.8532761838
47.8532761991
47.8532761971
47.8532763081
47.8532761905
47.8532761756
47.8532761833
47.853276196
47.853276178
47.8532762347
47.8532761564
47.8532761837
47.8532761902
47.8532761738
47.8532761795
26.3105898071
26.3105898071
26.3105898426
26.3105898903
26.3105899052
26.3105897474
26.3105897334
26.3105898956
26.3105897354
26.3105897581
26.3105898393
26.3105898313
26.3105897616
26.31058981
26.3105897737
26.3105896701
7.84825310261
7.84825310261
7.84825317584
7.84825315919
7.848253107
7.84825309327
7.84825298832
7.84825323572
7

## Dumping the calibration matrices

In [40]:
list_calibr = extract_transforms(result.x)

In [41]:
A = [list_calibr[0][0].tolist(), list_calibr[0][1].tolist()]
B = [list_calibr[1][0].tolist(), list_calibr[1][1].tolist()]

In [42]:
rospy.set_param("/vrep/calibration_matrix", A)

In [43]:
with open(rospack.get_path("reba_optim")+"/config/calibration_matrix_vrep.yaml", 'w') as f:
    yaml.dump(A, f)

In [44]:
with open(rospack.get_path("reba_optim")+"/config/calibration_matrix_tip.yaml", 'w') as f:
    yaml.dump(B, f)

## Testing calibration quality

In [4]:
with open(rospack.get_path("reba_optim")+"/config/calibration_matrix_tip.yaml") as f:
    calibration_matrix_tip = yaml.load(f)

In [5]:
calibration_matrix_tip

[[2.4566048961612835e-05, 4.6216949492736585e-07, -0.05986579235527341],
 [-1.9280322101558585e-07,
  -1.212167258258248e-06,
  -0.7071070513113319,
  0.7071065076104273]]

In [6]:
A = transformations.inverse_transform(calibration_matrix_tip)

In [8]:
calibration_matrix_tip = [map(float, A[0]), map(float, A[1].tolist())]

In [10]:
with open(rospack.get_path("reba_optim")+"/config/calibration_matrix_tip.yaml", 'w') as f:
    yaml.dump(calibration_matrix_tip, f)